# Exercise 2.1 - Train and Explore Word Vectors

# Working with word vectors
What's the NBA of baseball? One way to answer this is to form an analogy: basketball is to NBA as baseball is to ... ? The answer is MLB (Major League Baseball). One of the more interested developments in the past decade for NLP has been new methods for learning word vectors that encode both the meaning of words _and_ relational knowledge to solve these kinds of analogy tests. In this notebook, you'll see how these work for yourself!

The homework is broken up in several steps of preprocessing code (a constant fact of life for NLP), training a [word2vec](https://en.wikipedia.org/wiki/Word2vec) model using the [gensim](https://radimrehurek.com/gensim_3.8.3/models/word2vec.html) library and then exploring what the vectors have learned. For this notebook, you'll only train on a small set of the data to keep it quick (though this still learns a lot, as you'll see!); we've pre-computed a word2vec model for you on the full data as well, which you will use later in your evaluations. If you want to read up a bit more on word2vec, there's many great [blog](https://israelg99.github.io/2017-03-23-Word2Vec-Explained/) [posts](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) and you could check out the [original paper](https://arxiv.org/pdf/1301.3781.pdf)

This notebook will show you how to train your own vectors in the future too. Methods that learn distributional word vectors (like word2vec, LSA, or even counting) depend on the corpus they're trained on. Here, we're using our list of Wikipedia biographies. This corpus is quite rich in people, places, occupations, and all the things that people do. What all might a model learn from this? You'll see some of it in the tests we have prepared for you, but the model will know a lot more than what we've shown. Once you finish the core tasks, in your own exploration see what else the model has learned. Are there new types of analogies it encodes?

As usual in NLP, there are many options and hyperparameters to choose for word2vec. We've chosen a few to show you here (e.g., tokenization options) to help you get started exploring the space. You can try training on more data, adjust the window size, or the minimum token frequency to see how these impact performance and what the model learns. If you discover something interesting, feel free to discuss!

Finally, if you're feeling ambitious, try plotting pairs of words that have the same relationship, e.g., "NBA", "basketball", "MLB" and "baseball" and see if they have a geometric  relationship. As an example of this type of plot, see the plots for the word2vec alternative [glove](https://nlp.stanford.edu/projects/glove/) that also encodes this type of information. For plotting, try showing the first two principle components of the vectors (you'll first need to run PCA) or use [t-SNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html). If you make any interesting plots or discover something cool, please feel free to share in Slack!

In [1]:
import gensim
import gzip
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle
import re
import pandas as pd
from tqdm.notebook import tqdm
from gensim.models.word2vec import Word2Vec
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.test.utils import datapath

In [2]:
RANDOM_SEED = 655

To reduce the burden on memory, we use only the first 10,000 biographies and save them into a list that we'll call `bios`. We've pre-saved them as a pickle object `bios.p` so that you can load it directly.

In [3]:
bios = []
with open('assets/bios.p', 'rb') as f:
    bios = pickle.load(f)

Use nltk `word_tokenize` to split the biographies in `bios` into words and each bio's words as a separate list into a list called `nltk_tokenized_bios`. Try wrapping these in a `tqdm` call to see how long it takes

In [60]:
# bios[:1]

In [4]:
nltk_tokenized_bios = []
for bio in tqdm(bios):
    tokens = word_tokenize(bio)
    nltk_tokenized_bios.append(tokens)

Let's compare NLTK's tokenization with a regular expression-based extraction. Write a regular expression to find all sequences of word characters (`\w+`) for each biography in `bios` into words and each bio's words as a separate list into a list called `re_tokenized_bios`. Try wrapping these in a `tqdm` call to see how long it takes. 

In [5]:
re_tokenized_bios = []

pattern = re.compile(r'\w+')

for bio in tqdm(bios):
    tokens = pattern.findall(bio)
    re_tokenized_bios.append(tokens)

In [54]:
re_tokenized_bios[:1]

[['Early',
  'life',
  'Abraham',
  'Lincoln',
  'was',
  'born',
  'on',
  'February',
  '12',
  '1809',
  'the',
  'second',
  'child',
  'of',
  'Thomas',
  'Lincoln',
  'and',
  'Nancy',
  'Hanks',
  'Lincoln',
  'in',
  'a',
  'one',
  'room',
  'log',
  'cabin',
  'on',
  'Sinking',
  'Spring',
  'Farm',
  'near',
  'Hodgenville',
  'Kentucky',
  'He',
  'was',
  'a',
  'descendant',
  'of',
  'Samuel',
  'Lincoln',
  'an',
  'Englishman',
  'who',
  'migrated',
  'from',
  'Hingham',
  'Norfolk',
  'to',
  'its',
  'namesake',
  'Hingham',
  'Massachusetts',
  'in',
  '1638',
  'The',
  'family',
  'then',
  'migrated',
  'west',
  'passing',
  'through',
  'New',
  'Jersey',
  'Pennsylvania',
  'and',
  'Virginia',
  'Lincoln',
  's',
  'paternal',
  'grandparents',
  'his',
  'namesake',
  'Captain',
  'Abraham',
  'Lincoln',
  'and',
  'wife',
  'Bathsheba',
  'née',
  'Herring',
  'moved',
  'the',
  'family',
  'from',
  'Virginia',
  'to',
  'Jefferson',
  'County',
  'Ken

A surprising speed difference! Let's count how many unique tokens we found from each. Use a `Counter` to count the number of unique words for each and call these `nltk_word_counts` and `re_word_counts` 

In [53]:
nltk_tokenized_bios[:1]

[['Early',
  'life',
  'Abraham',
  'Lincoln',
  'was',
  'born',
  'on',
  'February',
  '12',
  ',',
  '1809',
  ',',
  'the',
  'second',
  'child',
  'of',
  'Thomas',
  'Lincoln',
  'and',
  'Nancy',
  'Hanks',
  'Lincoln',
  ',',
  'in',
  'a',
  'one-room',
  'log',
  'cabin',
  'on',
  'Sinking',
  'Spring',
  'Farm',
  'near',
  'Hodgenville',
  ',',
  'Kentucky',
  '.',
  'He',
  'was',
  'a',
  'descendant',
  'of',
  'Samuel',
  'Lincoln',
  ',',
  'an',
  'Englishman',
  'who',
  'migrated',
  'from',
  'Hingham',
  ',',
  'Norfolk',
  ',',
  'to',
  'its',
  'namesake',
  ',',
  'Hingham',
  ',',
  'Massachusetts',
  ',',
  'in',
  '1638',
  '.',
  'The',
  'family',
  'then',
  'migrated',
  'west',
  ',',
  'passing',
  'through',
  'New',
  'Jersey',
  ',',
  'Pennsylvania',
  ',',
  'and',
  'Virginia',
  '.',
  'Lincoln',
  "'s",
  'paternal',
  'grandparents',
  ',',
  'his',
  'namesake',
  'Captain',
  'Abraham',
  'Lincoln',
  'and',
  'wife',
  'Bathsheba',
  '(

In [8]:
nltk_word_counts = Counter()
re_word_counts = Counter()

for bio in nltk_tokenized_bios:
    for word in bio:
        nltk_word_counts[word] += 1

for bio in re_tokenized_bios:
    for word in bio:
        re_word_counts[word] += 1

In [9]:
len(nltk_word_counts), len(re_word_counts)

(294549, 219293)

In [10]:
#hidden tests are within this cell

Let's see what common words we missed through the imprecise regex matching. Create another `Counter` that contains the counts of all words not in `re_word_counts` but `nltk_word_counts` and call this `unique_word_counts`

HINT: familiarize with how to use “Counter” from collections. 
Documentation: https://docs.python.org/3/library/collections.html

In [11]:
unique_word_counts = Counter()

for word, count in nltk_word_counts.items():
    if word not in re_word_counts:
        unique_word_counts[word] = count

Print what are the 20 most common NLTK-unique words

In [12]:
unique_word_counts.most_common(20)

[(',', 887165),
 ('.', 582162),
 ("''", 248064),
 ('``', 236619),
 ("'s", 138914),
 (')', 93269),
 ('(', 93204),
 (':', 28415),
 (';', 26840),
 ("'", 20367),
 ('===', 9769),
 ('–', 7365),
 ('$', 5738),
 ('...', 5621),
 ("n't", 5528),
 ('*', 4844),
 ('&', 3940),
 ('!', 3296),
 ('U.S.', 3165),
 ('St.', 2727)]

In [13]:
len(unique_word_counts)

81881

In [14]:
#hidden tests are within this cell

Interesting! Looks like we're mostly missing punctuation and a few abbreviations. In this case, we can probably get away with tokenizing with just a regex without missing too much. 

For simplicitly, let's treat the regex-based token as our final solution and label this as a list called `all_tokenized_bios` and call the `re_word_counts` as just `word_counts`

In [15]:
all_tokenized_bios = re_tokenized_bios 
word_counts = re_word_counts

In [16]:
len(word_counts)

219293

In [17]:
#hidden tests are within this cell

Since we're dealing with lots of named entities, we didn't lower-case anything when tokenizing, which lets us potentially separate out "Apple" (the company) from "apple" (the fruit). Did we add many tokens from doing this? Create another `Counter` called `lowercase_word_counts` that records all the lower-case word counts and print the number of unique words. How many new upper-case words did we learn?

In [18]:
lowercase_word_counts = Counter()

for bio_tokens in all_tokenized_bios:
    for token in bio_tokens:
        lowercase_token = token.lower()
        lowercase_word_counts[lowercase_token] += 1

In [19]:
len(word_counts), len(lowercase_word_counts)

(219293, 191199)

In [20]:
#hidden tests are within this cell

In [21]:
%%time
len([w for w,c in word_counts.items() if c >= 100])

CPU times: user 30.4 ms, sys: 3.88 ms, total: 34.3 ms
Wall time: 36.6 ms


10815

Let's quickly train a word2vec model on the 10,000 tokenized biographies in `all_tokenized_bios`. This should train relatively quickly (~1 minute) and will let us do a few quick sanity tests. Call this model `quick_model` since it should be relative quick to train compared with training on the full set of biographies you have been using.

Here, we'll use arguments to specify:
* 50-dimensional vectors
* a window size of +/-2
* a minimum word frequency of 100
* 4 threads to process in parallel
* The class's `RANDOM_SEED`

*Note:* in the gensim that is installed for this homework, the vector size is specified as `size` in the argument. (Some other versions of the library have different names for the argument!)

In [22]:
%%time
from gensim.models import Word2Vec

# Train word2vec model
quick_model = Word2Vec(all_tokenized_bios, size=50, window=2, min_count=100, workers=4, seed=RANDOM_SEED)

CPU times: user 2min 2s, sys: 873 ms, total: 2min 3s
Wall time: 38 s


The word vectors are easily accessed through the `.wv` field of our model.

In [23]:
quick_word_vectors = quick_model.wv

In [24]:
quick_word_vectors

In [25]:
quick_word_vectors.similar_by_word("chemistry")[:10]

[('physics', 0.8876646161079407),
 ('psychology', 0.8781810402870178),
 ('biology', 0.8579720854759216),
 ('medicine', 0.8555915951728821),
 ('astronomy', 0.852459192276001),
 ('mathematics', 0.8478284478187561),
 ('economics', 0.8311997652053833),
 ('linguistics', 0.8291898965835571),
 ('sciences', 0.82292640209198),
 ('sociology', 0.8174623250961304)]

In [26]:
quick_word_vectors.similar_by_word("football")[:10]

[('baseball', 0.8867260217666626),
 ('basketball', 0.8860559463500977),
 ('tennis', 0.8519827723503113),
 ('hockey', 0.8467777371406555),
 ('soccer', 0.8452033400535583),
 ('cricket', 0.8333151340484619),
 ('golf', 0.8025889992713928),
 ('wrestling', 0.7965357899665833),
 ('chess', 0.794206976890564),
 ('rugby', 0.757988452911377)]

### Load the Word2Vec model trained on our Wikipedia corpus

Training a model on the full data can take 20-60 minutes so we've precomputed a model for you at `assets/wikipedia.100.word-vecs.kv`. Let's load these vectors using `gensim.models.KeyedVectors.load` and call this `full_word_vectors`. 

In [27]:
from gensim.models import KeyedVectors

full_word_vectors = KeyedVectors.load("assets/wikipedia.100.word-vecs.kv")

As a quick sanity check, print out the 10 most similar words for "chemistry" and "football" again using these new word vectors

In [28]:
full_word_vectors.similar_by_word("the")[:10]

[('its', 0.8000361323356628),
 ('their', 0.6638364195823669),
 ('this', 0.6563833951950073),
 ('our', 0.6386458873748779),
 ('his', 0.6207382678985596),
 ('The', 0.6133779287338257),
 ('a', 0.6116929054260254),
 ('another', 0.5941140651702881),
 ('her', 0.5800748467445374),
 ('every', 0.5692095160484314)]

In [29]:
full_word_vectors.similar_by_word("football")[:10]

[('soccer', 0.8991495370864868),
 ('basketball', 0.8783006072044373),
 ('baseball', 0.8465267419815063),
 ('hockey', 0.8135024309158325),
 ('lacrosse', 0.759275496006012),
 ('volleyball', 0.7534387111663818),
 ('rugby', 0.7447830438613892),
 ('softball', 0.7415612936019897),
 ('tennis', 0.7191575765609741),
 ('futsal', 0.7188178300857544)]

## Similarity and Relatedness

Looks like both models are capturing a surprising amount of information. Let's test how well both our quickly-learned and fully-trained vectors reflect human judgments of similarity and relatedness. Here, we'll use the [SimLex-999](https://fh295.github.io/simlex.html) and [WordSim-353](http://alfonseca.org/eng/research/wordsim353.html) benchmarks. The WordSim-353 dataset is notable for including judgments of both similarity and relatedness. 

To test on these datasets, we'll use the `evaluate_word_pairs` function of the `KeyedVectors` class which knows how to read these files and score them using both Pearson and Spearman's correlations. The datasets are stored at:
* `assets/wordsim_similarity_goldstandard.txt`
* `assets/wordsim_relatedness_goldstandard.txt`
* `assets/SimLex-999.tsv`

To start, let's compute similarities for `full_word_vectors` and `quick_word_vectors` on both SimLex999 and WordSim353. Save the results of each as variables named with the prefix and dataset, e.g., `full_simlex999` or `quick_wordsim353`, and print the results.

In [30]:
# YOUR CODE HERE
# raise NotImplementedError()
# Compute similarities using evaluate_word_pairs()
# Evaluate the models using SimLex-999
# Compute similarities using evaluate_word_pairs()
# Evaluate the models using SimLex-999
# full_simlex999 = full_word_vectors.evaluate_word_pairs('assets/SimLex-999.tsv', dummy4unknown=True)
# quick_simlex999 = quick_word_vectors.evaluate_word_pairs('assets/SimLex-999.tsv', dummy4unknown=True)
full_simlex999 = full_word_vectors.evaluate_word_pairs('assets/SimLex-999.tsv')
quick_simlex999 = quick_word_vectors.evaluate_word_pairs('assets/SimLex-999.tsv')
print("SimLex-999 results:")
print("Full word vectors:", full_simlex999)
print("Quick word vectors:", quick_simlex999)

# Evaluate the models using WordSim-353 Similarity
# full_wordsim353sim = full_word_vectors.evaluate_word_pairs('assets/wordsim_similarity_goldstandard.txt', dummy4unknown=True)
# quick_wordsim353sim = quick_word_vectors.evaluate_word_pairs('assets/wordsim_similarity_goldstandard.txt', dummy4unknown=True)
full_wordsim353 = full_word_vectors.evaluate_word_pairs('assets/wordsim_similarity_goldstandard.txt')
quick_wordsim353 = quick_word_vectors.evaluate_word_pairs('assets/wordsim_similarity_goldstandard.txt')

print("WordSim-353 Similarity results:")
print("Full word vectors:", full_wordsim353)
print("Quick word vectors:", quick_wordsim353)

# # Evaluate the models using WordSim-353 Relatedness
# full_wordsim353rel = full_word_vectors.evaluate_word_pairs('assets/wordsim_relatedness_goldstandard.txt', dummy4unknown=True)
# quick_wordsim353rel = quick_word_vectors.evaluate_word_pairs('assets/wordsim_relatedness_goldstandard.txt', dummy4unknown=True)
# print("WordSim-353 Relatedness results:")
# print("Full word vectors:", full_wordsim353rel)
# print("Quick word vectors:", quick_wordsim353rel)


SimLex-999 results:
Full word vectors: ((0.23603218778235258, 4.7368615995633334e-14), SpearmanrResult(correlation=0.24304544269746545, pvalue=7.858972973411315e-15), 0.5005005005005005)
Quick word vectors: ((0.2231426900203602, 8.77186124729192e-08), SpearmanrResult(correlation=0.2145946720496291, pvalue=2.739942706236654e-07), 43.64364364364364)
WordSim-353 Similarity results:
Full word vectors: ((0.4919819225840672, 1.377796887179771e-13), SpearmanrResult(correlation=0.4618989457903368, pvalue=5.796686483312412e-12), 1.477832512315271)
Quick word vectors: ((0.5426167183480762, 6.291471795033972e-11), SpearmanrResult(correlation=0.5187994774344606, pvalue=5.720840504161241e-10), 38.42364532019704)


In [31]:
#hidden tests are within this cell

Let's do the same for relatedness! Here, call your variables `full_relatedness353` and `quick_relatedness353`

In [32]:
# Load relatedness dataset
full_word_vectors = KeyedVectors.load('assets/wikipedia.100.word-vecs.kv')
# Compute relatedness for full_word_vectors and quick_word_vectors on WordSim353
full_relatedness353 = full_word_vectors.evaluate_word_pairs('assets/wordsim_relatedness_goldstandard.txt')
quick_relatedness353 = quick_model.wv.evaluate_word_pairs('assets/wordsim_relatedness_goldstandard.txt')

In [33]:
#hidden tests are within this cell

## Word Analogies
Word vectors can not only capture similarity in meaning but relational structure as well. Here, we'll look for analogies of the for `a:b::c:d` or "a is to b as c is to d". In solving these types of analogies, we'll use the `most_similar` function of `KeyedVectors` that will do the vector arithmetic, as described in the original word2vec  paper.

In your first task, write a function `get_analogy` that takes arguments `a`, `b`, `c`, and uses the `most_similar` function to find and return the `d` item that is analogous to `b` in the `full_word_vectors` space.

*NOTE:* you should make sure to pass lists to `most_similar`, not tuples (the latter will work but give incorrect results)

In [34]:
# def get_analogy(a, b, c, full_word_vectors):
#     """
#     Finds the word that is analogous to b in the full_word_vectors space,
#     based on the analogy a:b :: c:d.
    
#     Args:
#     - a (str): The first word in the analogy (e.g., "man").
#     - b (str): The second word in the analogy (e.g., "king").
#     - c (str): The third word in the analogy (e.g., "woman").
#     - full_word_vectors (Word2VecKeyedVectors): The Word2VecKeyedVectors object
#       containing the full set of word vectors.
      
#     Returns:
#     - d (str): The word that is analogous to b in the full_word_vectors space.
#     """
#     # Calculate the vector difference between a and b, and add it to c
#     a_vec = full_word_vectors[a]
#     b_vec = full_word_vectors[b]
#     c_vec = full_word_vectors[c]
#     d_vec = c_vec + (b_vec - a_vec)
    
#     # Find the most similar word to the resulting vector
#     d = full_word_vectors.most_similar([d_vec], topn=1)[0][0]
    
#     return d

def get_analogy(a, b, c, full_word_vectors=full_word_vectors):
    result = full_word_vectors.most_similar(positive=[c, b], negative=[a])
    return result[0][0]

### Task: Find the appropriate analogy entities/concepts for the following relations

For all questions below, please return your answer as a single `str`. 

### Relation 1: Country::Capital

Find the capital of each following countries, according to the fact that London is the capital of UK:
1. France
2. Germany
3. Italy
4. Austria
5. Denmark

In [35]:
answer = get_analogy('UK', 'London', 'France')
print(answer)
#hidden tests are within this cell

Paris


In [36]:
answer = get_analogy('UK', 'London', 'Italy')
print(answer)
#hidden tests are within this cell

Rome


In [37]:
answer = get_analogy('UK', 'London', 'Germany')
print(answer)
#hidden tests are within this cell

Berlin


In [38]:
answer = get_analogy('UK', 'London', 'Austria')
print(answer)
#hidden tests are within this cell

Vienna


In [39]:
answer = get_analogy('UK', 'London', 'Denmark')
print(answer)
#hidden tests are within this cell

Copenhagen


Well, pretty amazing! Considering that these are only trained on Wikipedia biographies, the model has captured quite a bit of information about places.

### Relation 2: Association::Sports

There are a lot of athletes in our dataset. Let's see if we can recover the relationship between sports associations and the sports played in them, NBA is to basketball as NHL is to ???
1. NFL - National ????? League
2. NHL - National ????? League
3. MLB - Major League ?????
4. MLS - Major League ?????
5. NLL - National ????? League

In [40]:
answer = get_analogy('NBA', 'basketball', 'NHL')
print(answer)
#hidden tests are within this cell

soccer


In [41]:
answer = get_analogy('NBA', 'basketball', 'MLB')
print(answer)
#hidden tests are within this cell

baseball


In [42]:
answer = get_analogy('NBA', 'basketball', 'MLS')
print(answer)
#hidden tests are within this cell

soccer


In [43]:
answer = get_analogy('NBA', 'basketball', 'NFL')
print(answer)
#hidden tests are within this cell

baseball


In [44]:
answer = get_analogy('NBA', 'basketball', 'NLL')
print(answer)
#hidden tests are within this cell

soccer


What if we try a different one to predict for the NFL?

In [45]:
answer = get_analogy('MLB', 'baseball', 'NFL')
print(answer)
#hidden tests are within this cell

basketball


What if we try the reverse?

In [46]:
answer = get_analogy('NFL', 'football', 'NBA')
print(answer)
#hidden tests are within this cell

soccer


Huh! This suggests that NFL in our dataset is occurring in different contexts than other sports leagues. But it can be tough to tell what's fully responsible! Perhaps the use of football as both "soccer" and "gridiron football" causes an issue? The model sure seems to like "soccer". 

In [47]:
for bio in all_tokenized_bios[:1000]:
    for i, w in enumerate(bio):
        if w == 'NFL':
            print(' '.join(bio[i-4:i+5]))

Spears performing at the NFL Kickoff Live in September
worked with NBA and NFL teams inspiring athletes to
and Walsh two future NFL Hall of Famers would
me out of the NFL In 1977 Walsh was
coach again in the NFL There were at least
the rights to an NFL franchise but Bell disparaged
Bell was let go NFL career Philadelphia Eagles 1933
s opinion on the NFL had changed and he
being advised by the NFL that a prerequisite to
the winner of the NFL championship game with the
is second lowest in NFL history to only Phil
1943 40 of the NFL rosters had been drafted
1944 to displace the NFL s sovereignty in professional
situation in its history NFL commissioner 1946 1959 Election
Hapes Filchock and the NFL schedule 1946 1948 Elmer
was appointed the first NFL commissioner in 1941 but
of contention among the NFL owners since the league
suspended At the next NFL owners meeting Bell was
to permanently ban any NFL associated personnel for betting
the payroll of the NFL to investigate potential betting


### Relation 3: Expert::Field

Find what fields do the following experts have expertise in, according to the fact that biologists have expertise in biology:
1. sociologists
2. psychologists
3. neuroscientists
4. zoologists
5. physiologists

In [48]:
answer = get_analogy('biologists', 'biology', 'chemists')
print(answer)
#hidden tests are within this cell

chemistry


In [49]:
answer = get_analogy('biologists', 'biology', 'oncologist')
print(answer)
#hidden tests are within this cell

oncology


In [50]:
answer = get_analogy('biologist', 'biology', 'artist')
print(answer)
#hidden tests are within this cell

art


In [51]:
answer = get_analogy('biologist', 'biology', 'zoologist')
print(answer)
#hidden tests are within this cell

botany


In [52]:
answer = get_analogy('biologists', 'biology', 'archaeologists')
print(answer)
#hidden tests are within this cell

archaeology


Not too bad! It seems to have learned something about what the fields and people do!

## Reflection

Which tasks surprised you more? These results show that our model knows a surprising amount. What else does it know? Feel free to try out new types of relationships and words. Keep in mind that it only knows about information in relationship to people since we've trained it on biographies. 

One thing we didn't try was training our model with multi-word expressions (MWEs). Gensim actually supports finding these for us using its `Phrases` package when processing the corpus and learning word vectors. This would let us compare things like `United States` as a phrase. Which kinds of MWEs would be fun to test out? Feel free to retrain the model here and test out new analogies on MWEs; if you find interest ones, post on course Slack.